link for this kaggle kernel: https://www.kaggle.com/nadergo/hybrid-anime-recommendations-with-pytorch

The goal is to make a deep learning model that will be used in a hybrid recommendation system 

embedding layers will help us with that a lot (basically a layer that maps an index to a vector of trainable  weights) 

I was inspired by this talk, it's really helpful
https://www.youtube.com/watch?v=HG3FDCegKVc

![the neural network](https://i.ibb.co/HXyf49X/recommendation.png)

this is the architecture of the neural network

In [1]:
import os # this makes better error tracing 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.metrics import mean_squared_error


In [3]:
from torch import nn
import torch
from torch.nn  import functional as F
import torch.optim as  optim 
if torch.cuda.is_available():  
  dev = "cuda:0" 
  print("gpu up")
else:  
  dev = "cpu"  
device = torch.device(dev)

gpu up


In [4]:
ratings = pd.read_csv("/kaggle/input/anime-recommendations-database/rating.csv")
anime = pd.read_csv("/kaggle/input/anime-recommendations-database/anime.csv")

Anime dataframe  

    anime_id - myanimelist.net's unique id identifying an anime.
    name - full name of anime.
    genre - comma separated list of genres for this anime.
    type - movie, TV, OVA, etc.
    episodes - how many episodes in this show. (1 if movie).
    rating - average rating out of 10 for this anime.
    members - number of community members that are in this anime's
    "group".


Ratings dataframe 

    user_id - non identifiable randomly generated user id.
    anime_id - the anime that this user has rated.
    rating - rating out of 10 this user has assigned (-1 if the user watched it but didn't assign a rating).
    
      
(actually this description from the dataset page)

In [5]:
ratings

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [6]:
anime

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [7]:
anime.nunique() # number of unique values in each column

anime_id    12294
name        12292
genre        3264
type            6
episodes      187
rating        598
members      6706
dtype: int64

In [8]:
anime.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [9]:
anime.eq("Unknown").sum()

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


anime_id      0
name          0
genre         0
type          0
episodes    340
rating        0
members       0
dtype: int64

the anime df has some few null values but dropping these rows might affect the performance greatly as we should take into considerations that we will need the combinations between animes and users

In [10]:
"""
we will replace null values from categorical columns with "NULL"(like it's a new category) 
and for the rating column we will use the mean
"""
anime.genre.fillna("NULL",inplace=True)
anime.type.fillna("NULL",inplace=True)
anime.rating.fillna(anime.rating.mean(),inplace=True)


In [11]:
"""
these are the animes that we don't have information about, they are just 3 and
they have only 10 combinations(which means they contribute with only 10 samples) with the users but they will produce null values  for other features  
we will drop them  
"""
badAnimes = [x  for x in ratings.anime_id.unique() if x not in anime.anime_id.unique() ] 
badIndecies = ratings[ratings.anime_id.isin(badAnimes) ].index
ratings.drop(badIndecies,inplace=True)

In [12]:
ratings.rating.eq(-1).sum() # -1 means the user didn't rate this anime

1476488

Doing some preprocessing

In [13]:
"""
on how weird this processing seems, but it had to be like that for memory efficiency

"""

ratingsMean = anime.rating.mean()
membersMean = anime.members.mean()
base = anime.copy() 

base.drop(["episodes","name"],axis=1,inplace=True) #dropped episodes  many null values 
# and will cause a false  relationship as there are many movies (1 episode)
base.rename(columns={"rating":"avgRating"},inplace=True)

base.genre = base.genre.apply(lambda x: " ".join(x.split(" ")).split(", ")) #to remove unnecessary spaces
genres = [] # generating the genres data
for i in range(len(base)):
    for g in base.genre[i]:
        if g in genres:
            base["genre_"+g][i] = 1
        else:
            base["genre_"+g] = 0
            base["genre_"+g][i] = 1
            genres.append(g)



base = pd.concat([base,pd.get_dummies(base['type'], prefix='type',dummy_na=True)],axis=1).drop(['type'],axis=1)


ratings =ratings[["rating","user_id","anime_id"]] # rearranging columns
base= pd.merge(ratings,base,how="left",on="anime_id")
ratings = None


base.avgRating.fillna(ratingsMean,inplace=True)
base.members.fillna(membersMean,inplace=True)
base.drop(["genre"],axis=1,inplace=True)


base.fillna(0,inplace=True) # for any 






/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
import random
random.seed(42)

"""
we will set the productionData to the data where the users didn't rate the anime
and we will use 500000 rows for validation

"""


productionData= base[base.rating == -1]
base.drop(base[base.rating == -1].index,inplace=True)
base = base.sample(frac=1).reset_index(drop=True)
validation = base.iloc[:500000]
train = base.iloc[500000:]
base = None # for reducing memory usage (we won't need this df again)


In [15]:
membersEnc = MinMaxScaler()
avgRatingEnc = MinMaxScaler()

animeEncoder = LabelEncoder()

"""
since the anime ids are not consecutive numbers, we will have to label encode them for the embedding layer 
"""

anime["anime_id"] = animeEncoder.fit_transform(anime["anime_id"])



train["anime_id"] = animeEncoder.transform(train["anime_id"])
train["avgRating"] = avgRatingEnc.fit_transform(train["avgRating"].to_numpy().reshape(-1,1))[:,0]
train["members"] = membersEnc.fit_transform(train["members"].to_numpy().reshape(-1,1))[:,0]

validation["anime_id"] = animeEncoder.transform(validation["anime_id"])
validation["avgRating"] = avgRatingEnc.transform(validation["avgRating"].to_numpy().reshape(-1,1))[:,0]
validation["members"] = membersEnc.transform(validation["members"].to_numpy().reshape(-1,1))[:,0]


productionData["anime_id"] = animeEncoder.transform(productionData["anime_id"])
productionData["avgRating"] = avgRatingEnc.transform(productionData["avgRating"].to_numpy().reshape(-1,1))[:,0]
productionData["members"] = membersEnc.transform(productionData["members"].to_numpy().reshape(-1,1))[:,0]



In [16]:
"""
finally the neural network 

"""

class RecommendationNet(nn.Module):
    def __init__(self):
        super(RecommendationNet,self).__init__()
        self.users = nn.Embedding(73517,100) 
        self.animes = nn.Embedding(12294,100)
        self.linear1 = nn.Linear(100+100+54,128)
        self.linear2 = nn.Linear(128,32)
        self.linear3 = nn.Linear(32,1)
    def forward(self,x):
        user = x[:,0].long() # here am selecting the user and anime ids from the input 
        anime = x[:,1].long() 
        otherfeatures = x[:,2:]
        userVector = self.users(user)
        animeVector = self.animes(anime)
#         print(userVector.shape,animeVector.shape,otherfeatures.shape) was used for debugging 
        layer1 = torch.cat((userVector,animeVector,otherfeatures),1)# concatenating vectors
        layer2 = F.relu(self.linear1(layer1))
        layer3 = F.relu(self.linear2(layer2))
        out = torch.sigmoid(self.linear3(layer3)) 
        return out
    

myNN = RecommendationNet()
myNN.to(device)
        
        
        
        

RecommendationNet(
  (users): Embedding(73517, 100)
  (animes): Embedding(12294, 100)
  (linear1): Linear(in_features=254, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=32, bias=True)
  (linear3): Linear(in_features=32, out_features=1, bias=True)
)

In [17]:
optimizer = optim.Adagrad(myNN.parameters(),lr = 0.001)

batch_size = 128

npData = train.to_numpy()
npData[:,:1] = npData[:,:1]/10 # scaling the target variable
# traintrues = np.expm1(npData[:,4].reshape(-1,1)).reshape(-1)
def ceil(a,b):
    return -(-a//b)

n_samples = len(npData)
better_batch_size = ceil(n_samples, ceil(n_samples, batch_size))

for i in range(10):
#     preds=[]
    for i in range(ceil(n_samples, better_batch_size)):
        batch = npData[i * better_batch_size: (i+1) * better_batch_size]
        batch = torch.Tensor(batch).to(device)
        X = batch[:,1:]
        y = batch[:,:1]
        myNN.zero_grad()
#         print(i)
        pred = myNN(X)
#         preds.extend(np.expm1(pred.cpu().detach().numpy()).reshape(-1))
        err = F.mse_loss(pred,y)
        err.backward()
        optimizer.step()
    print(torch.sqrt(err))
    valpreds = myNN(torch.Tensor(validation.to_numpy()[:,1:]).to(device)).cpu().detach().numpy().reshape(-1)
    print(np.sqrt(mean_squared_error(validation.rating.to_numpy(),valpreds*10)),"Validation Error")


tensor(0.1771, device='cuda:0', grad_fn=<SqrtBackward>)
1.461207320974091 Validation Error
tensor(0.1763, device='cuda:0', grad_fn=<SqrtBackward>)
1.4432169557988868 Validation Error
tensor(0.1755, device='cuda:0', grad_fn=<SqrtBackward>)
1.4323484569846707 Validation Error
tensor(0.1746, device='cuda:0', grad_fn=<SqrtBackward>)
1.42408668254435 Validation Error
tensor(0.1737, device='cuda:0', grad_fn=<SqrtBackward>)
1.4171481539198654 Validation Error
tensor(0.1728, device='cuda:0', grad_fn=<SqrtBackward>)
1.4110062442891336 Validation Error
tensor(0.1721, device='cuda:0', grad_fn=<SqrtBackward>)
1.4054091222655618 Validation Error
tensor(0.1714, device='cuda:0', grad_fn=<SqrtBackward>)
1.4002110815934319 Validation Error
tensor(0.1708, device='cuda:0', grad_fn=<SqrtBackward>)
1.3953341715937124 Validation Error
tensor(0.1703, device='cuda:0', grad_fn=<SqrtBackward>)
1.3907277185254183 Validation Error


* Now we can predict how a specific user will rate a specific anime with some level of confidence 

In [20]:
"""
we can use the productionData df(data where users didn't rate animes) to recommend animes for users
we will choose an arbitrary user id like 1 and will choose to show an arbitrary number also 
like the top 10 animes

"""

npRecommend = productionData[productionData.user_id == 1].to_numpy()
npRecommend[:,0] = myNN(torch.Tensor(npRecommend[:,1:]).to(device)).to(device).cpu().detach().numpy().reshape(-1)
indecies =  np.argsort(npRecommend[:,0])[-10:][::-1]
anime_ids = [npRecommend[i,2] for i in indecies]

recommendedAnimes = anime[anime.anime_id.isin(anime_ids)]
    
recommendedAnimes





,anime_id,name,genre,type,episodes,rating,members
29,1822,Tengen Toppa Gurren Lagann,"Action, Adventure, Comedy, Mecha, Sci-Fi",TV,27,8.78,562962
67,8722,Kiseijuu: Sei no Kakuritsu,"Action, Drama, Horror, Psychological, Sci-Fi, ...",TV,24,8.59,425457
86,7466,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229
118,8165,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291
122,6632,Kuroko no Basket,"Comedy, School, Shounen, Sports",TV,25,8.46,338315
131,3566,Toradora!,"Comedy, Romance, School, Slice of Life",TV,25,8.45,633817
159,4640,Angel Beats!,"Action, Comedy, Drama, School, Supernatural",TV,13,8.39,717796
219,9051,Yahari Ore no Seishun Love Comedy wa Machigatt...,"Comedy, Drama, Romance, School",TV,13,8.31,222994
251,1845,Darker than Black: Kuro no Keiyakusha,"Action, Mystery, Sci-Fi, Super Power",TV,25,8.25,440334
458,3876,Kuroshitsuji,"Action, Comedy, Demons, Fantasy, Historical, S...",TV,24,8.06,424919
